In [1]:
import pandas as pd
import numpy as np
import altair as alt
import locale

locale.setlocale(locale.LC_ALL, 'sv_SE')
%config InlineBackend.figure_format = 'retina'

In [2]:
location="https://scb.se/hitta-statistik/statistik-efter-amne/befolkning/befolkningens-sammansattning/befolkningsstatistik/pong/tabell-och-diagram/preliminar-statistik-over-doda/"
location="/Users/niklas/Downloads/2020-04-24---preliminar-statistik-over-doda-inkl-eng.xlsx"

raw_data = pd.read_excel(location, sheet_name="Tabell 1", skiprows=5, header=1, index_col=0, usecols=[0,1,2,3,4,5,6])
raw_data

,2015,2016,2017,2018,2019,2020
DagMånad,,,,,,
1 januari,295,245,339,275,300,254
2 januari,272,272,317,312,276,299
3 januari,281,293,310,295,271,256
4 januari,256,260,305,285,282,272
5 januari,268,260,304,291,275,265
...,...,...,...,...,...,...
28 december,274,262,273,261,215,0
29 december,292,274,276,263,244,0
30 december,283,307,275,232,226,0


In [122]:
data = raw_data.stack().reset_index()
data['Date'] = data['DagMånad'] + data['level_1'].apply(str)
data['Date'] = pd.to_datetime(data['Date'], format='%d %B%Y', errors='coerce')
data.rename(inplace=True, columns={'DagMånad': 'DayMonth', 'level_1': 'Year', 0: 'Dead'})

In [123]:
data

,DayMonth,Year,Dead,Date
0,1 januari,2015,295,2015-01-01
1,1 januari,2016,245,2016-01-01
2,1 januari,2017,339,2017-01-01
3,1 januari,2018,275,2018-01-01
4,1 januari,2019,300,2019-01-01
...,...,...,...,...
2197,Okänd dödsdag,2016,1963,NaT
2198,Okänd dödsdag,2017,2230,NaT
2199,Okänd dödsdag,2018,2513,NaT
2200,Okänd dödsdag,2019,2616,NaT


In [131]:
df_history = data[data['Date'] < '2020-01-01']
df_current = data['2020-01-01' <= data['Date']]
df_current = df_current[df_current['Date'] < '2020-04-20']

history = alt.Chart(
    df_history,
    height=600,
    width=800,
    title="Daily Deaths in Sweden 2015-2020"
).mark_point(
    color='#d9d9d9'
).encode(
    x='monthdate(Date)',
    y=alt.Y('Dead', title="Daily Deaths"),
)

mean = alt.Chart(df_history).mark_line(
    color='#969696',
    size=3
).transform_window(
    rolling_mean='mean(Dead)',
    frame=[-10, 10]
).encode(
    x='monthdate(Date)',
    y=alt.Y('rolling_mean:Q'),
)

current = alt.Chart(df_current).mark_point(
    color='#fdae6b',
).encode(
    x='monthdate(Date)',
    y='Dead',
)

current_mean = alt.Chart(df_current).mark_line(
    color='#e6550d',
    size=4
).transform_window(
    rolling_mean='mean(Dead)',
    frame=[-10, 10]
).encode(
    x='monthdate(Date)',
    y=alt.Y('rolling_mean:Q'),
    tooltip=['Dead', 'rolling_mean:Q']
)

history + mean + current + current_mean.interactive()

alt.LayerChart(...)